# TP ISI

### Importo Bibliotecas

In [1317]:
import pandas as pd

### Lectura de datos

In [1318]:
df = pd.read_csv('data/survey_results_public.csv')

### Columnas

In [1319]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64461 entries, 0 to 64460
Data columns (total 61 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Respondent                    64461 non-null  int64  
 1   MainBranch                    64162 non-null  object 
 2   Hobbyist                      64416 non-null  object 
 3   Age                           45446 non-null  float64
 4   Age1stCode                    57900 non-null  object 
 5   CompFreq                      40069 non-null  object 
 6   CompTotal                     34826 non-null  float64
 7   ConvertedComp                 34756 non-null  float64
 8   Country                       64072 non-null  object 
 9   CurrencyDesc                  45472 non-null  object 
 10  CurrencySymbol                45472 non-null  object 
 11  DatabaseDesireNextYear        44070 non-null  object 
 12  DatabaseWorkedWith            49537 non-null  object 
 13  D

### Cantidad de datos en jos sat

In [1320]:
df['JobSat'].value_counts()

Very satisfied                        14611
Slightly satisfied                    13930
Slightly dissatisfied                  7124
Neither satisfied nor dissatisfied     5783
Very dissatisfied                      3746
Name: JobSat, dtype: int64

### Respondent

In [1321]:
# Es el numero de encuestado, se elimina
df.drop(['Respondent'], axis=1, inplace=True)

### MainBranch

In [1322]:
df['MainBranch'].value_counts()

I am a developer by profession                                                   47193
I am a student who is learning to code                                            7970
I am not primarily a developer, but I write code sometimes as part of my work     5502
I code primarily as a hobby                                                       2314
I used to be a developer by profession, but no longer am                          1183
Name: MainBranch, dtype: int64

In [1323]:
# Nos quedamos como aquellos que son desarrolladores por profesion o, si bien no son desarrolladores, 
# escriben codigo como parte de su trabajo. Se descarta el resto ya que no aportan a la incestigación
# Luego se elimina la columna
values= ['I am a developer by profession', 'I am not primarily a developer, but I write code sometimes as part of my work']
df = df[df.MainBranch.isin(values)]
df['MainBranch'].value_counts()

I am a developer by profession                                                   47193
I am not primarily a developer, but I write code sometimes as part of my work     5502
Name: MainBranch, dtype: int64

In [1324]:
# Modificamos la columna para que sea más entendible
df.MainBranch = df.MainBranch.apply(lambda x: 'Developer' if x == 'I am a developer by profession' else 'Not developer')

### Hobbyist 

In [1325]:
### Describe si ademas de hacerlo de manera laboral, la persona programa como un hobby.
### Nos da una media del interes y el gusto de la persona en la programación. Valores Yes y No, se considera correcta

In [1326]:
df.Hobbyist.value_counts()

Yes    40433
No     12224
Name: Hobbyist, dtype: int64

### Age

In [1327]:
# Se consideran nulas todas aquellas edades menores a 16 años y mayores a 90 años
df.Age = df.Age.apply(lambda x: None if (x<16 or x>90) else x)

In [1328]:
def classify_age(x):
    if (x>=16 and x < 25):
        return 'Entre 16 y 25'
    if (x >= 25 and x < 35):
        return 'Entre 25 y 35'
    if (x >= 35 and x < 45):
        return 'Entre 35 y 45'
    if (x >= 45 and x < 55):
        return 'Entre 45 y 55'
    if (x >=55 and x < 65):
        return 'Entre 55 y 65'
    if (x >=65 and x < 75):
        return 'Entre 65 y 75'
    if (x >=75 and x < 85):
        return 'Entre 75 y 85'
    return None
    
# Vamos a dividir a las personas en tres grupos: 
df.Age = df.Age.apply(lambda x: classify_age(x))    


### Age1stCode

In [1329]:
# Edad en la que la persona escribió sus primeras lineas de código
# Esta columna nos sirve para ver si la persona aprendió a programar a una edad adulta o de pequeño
# Vamos a clasificarla en algunos grupos:

In [1330]:
df.Age1stCode=df.Age1stCode.replace('Older than 85', 65)
df.Age1stCode=df.Age1stCode.replace('Younger than 5 years', 5)
df.Age1stCode=df.Age1stCode.fillna(90)


In [1331]:
def classify_Age1stCode(x):
    if (x>=4 and x < 12):
        return 'Antes de los 12'
    if (x >= 12 and x < 18):
        return 'Entre los 12 y los 18'
    if (x >= 18 and x < 25):
        return 'Entre los 18 y los 25'
    if (x >= 25 and x < 35):
        return 'Entre los 25 y los 35'
    if (x >= 35 and x < 45):
        return 'Entre los 35 y los 45'
    if (x >= 45 and x < 60):
        return 'Entre los 45 y los 60'
    if (x >= 60 and x <= 85):
        return 'Despues de los 60'
    return None

# Vamos a dividir a las personas en tres grupos: 
df.Age1stCode = df.Age1stCode.apply(lambda x: classify_Age1stCode(int(x))) 

In [1332]:
df.Age1stCode.value_counts()

Entre los 12 y los 18    24914
Entre los 18 y los 25    12139
Antes de los 12           8881
Entre los 25 y los 35     1751
Entre los 35 y los 45      181
Entre los 45 y los 60       41
Despues de los 60           14
Name: Age1stCode, dtype: int64

### CompFreq

In [1333]:
# Frecuencia con la que se recibe la compensación, el campo está correcto

In [1334]:
df.CompFreq.value_counts()

Yearly     19818
Monthly    18899
Weekly      1352
Name: CompFreq, dtype: int64

### CompTotal

In [1335]:
# Es el salario pero expresado en moneda local por lo que se descarta ya que se utilizará el salario en dolares
df.drop(['CompTotal'], axis=1, inplace=True)

###  ConvertedComp 

In [1336]:
# Es el salario en dolares, esta columa nos interesa pero tambien vamos a discretizarla

In [1337]:
def classify_ConvertedComp(x):
    if (x == 0):
        return 'Gratis'
    if (x > 100 and x < 1000):
        return 'Menos de 1000'
    if (x > 1000 and x < 3000):
        return 'Entre 1.000 y los 3.000'
    if (x > 3000 and x < 5000):
        return 'Entre 3.000 y los 5.000'
    if (x > 5000 and x < 10000):
        return 'Entre 5.000 y los 10.000'
    if (x > 10000 and x < 20000):
        return 'Entre 10.000 y los 20.000'
    if (x > 20000 and x < 40000):
        return 'Entre 20.000 y los 40.000'
    if (x > 40000 and x < 60000):
        return 'Entre 40.000 y los 60.000'
    if (x > 60000 and x < 80000):
        return 'Entre 60.000 y los 80.000'
    if (x >= 80000 and x < 100000):
        return 'Entre 80.000 y los 100.000'
    if (x >= 100000 and x < 250000):
        return 'Entre 100.000 y los 250.000'
    if (x >= 250000 and x < 500000):
        return 'Entre 250.000 y los 500.000'
    if (x >= 500000 and x < 1000000):
        return 'Entre 500.000 y los 1.000.000'
    if (x >= 1000000):
        return 'Más de 1.000.000'
    return None

df.ConvertedComp = df.ConvertedComp.apply(lambda x: classify_ConvertedComp(x))

In [1338]:
df.ConvertedComp.value_counts()

Entre 100.000 y los 250.000      6300
Entre 20.000 y los 40.000        5957
Entre 40.000 y los 60.000        5713
Entre 60.000 y los 80.000        4342
Entre 10.000 y los 20.000        3506
Entre 80.000 y los 100.000       2980
Entre 5.000 y los 10.000         2143
Entre 3.000 y los 5.000           799
Entre 250.000 y los 500.000       675
Más de 1.000.000                  656
Entre 500.000 y los 1.000.000     584
Entre 1.000 y los 3.000           509
Gratis                            142
Menos de 1000                     139
Name: ConvertedComp, dtype: int64

### Country

In [1339]:
# Convert it to continent

In [1340]:
aux = df.Country.dropna()

In [1341]:
aux = aux.apply(lambda x: replace_value(x))

In [1342]:
def get_continent(x):
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(x)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        return country_continent_name
    except:
        return None
        

In [1343]:
aux  = aux.apply(lambda x: get_continent(x))

In [1344]:
df.Country = aux
df.rename(columns = {'Country':'Continent'},inplace=True)

In [1345]:
df['Continent'].value_counts()

Europe           20898
North America    13311
Asia             12517
South America     2655
Africa            1722
Oceania           1338
Name: Continent, dtype: int64